# Specs

- using user mvashishtha_test, account sfctest0, lage warehouse TESTWH_MVASHISHTHA, 

# Preamble

In [1]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvUZyzaEggbIuNAUd0ggwS4Nb8KWQbEtuTo5hvz6yHxkkodkOqMHjbR7u3d7g%2Bt9kVvPTAGXYohc20EWE7FMuNgOURTetHrIAk1FQnMp2BAdGKDr0QBokZfEr%2FROPLB%2FFQNtmUICSPMxRJUSRFLgQAQtGBAdk6V%2FOyOe7RAKwJQ2cuhMSYAbrZ3WJcG4rmu7bttSbbHnOA52%2BtigGsg39E6i%2FFqjVFLLWOYXyt709ImEi51OI2EQRmFxJv7g4jSCr1Q2JxCQaRguWov5MkSWf%2BluLAVUBVNLpp55zKKH2ckAGAfLu%2FlqOo%2BWP20Qsk5zmrFYFmWlTTXb3HDKEpzLLTczCiZDVGY8eYxS%2BvL4NPO9aZ9uNF3dh7veUzvOtOrdhxs1Xv%2B6jf7O6NUmi5H155Ko1yQaAFQsEE2O2jw53lXLaZsTul3itkmnb3e7nTWyJiZHLqg%2BMi9mG4sb%2FmLLTNOjOVqW%2BM03ZvusV6eQqsPvQHSC1dorv2MAiZuY0GlTyNGAGv1v%2FwP8nnVetjsz%2F2CykDmPD9aNVAXVn8fj2u7xhSet9AglrKA895NEMQATU57LeqwY1WantaoYwqOT6setHr0C&RelayState=ver%3A1-hi

# Use case 1: working with a single small table

In this example, we read the Snowhouse table `SAMPLE_DATA.TPCH_SF1.CUSTOMER`, which is 150k rows and 10.3 MB in Snowflake.

In [2]:
df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF1.CUSTOMER")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_RATZE1EF75' for snapshotting. DataFrame creation might take some time.


Just printing the data is visibly slow...

In [4]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...
3,30004,Customer#000030004,tkR93ReOnf9zYeO,23,33-870-136-4375,3308.55,AUTOMOBILE,ssly bold deposits. final req
4,30005,Customer#000030005,pvq4uDoD8pEwpAE01aesCtbD9WU8qmlsvoFav5,9,19-144-468-5416,-278.54,MACHINERY,ructions behind the pinto beans x-ra
...,...,...,...,...,...,...,...,...
149995,29996,Customer#000029996,BnZVGZiAgcEImNm9iD,7,17-536-308-8025,4035.17,FURNITURE,"ual instructions. bold, silent foxes nag blith..."
149996,29997,Customer#000029997,lTbDYXdQ74JctD UbRbXCqF2b8,9,19-631-777-4123,2015.90,HOUSEHOLD,eodolites detect slyly alongside of the quickl...
149997,29998,Customer#000029998,ZxxiuDruzi98CcymR,23,33-619-315-9722,-810.56,FURNITURE,xpress packages. accounts sleep carefully iron...
149998,29999,Customer#000029999,CuPA4UpgTCYiXrBrpiSO D,12,22-824-951-8333,3865.14,FURNITURE,eposits-- accounts haggle across the slyly per...


and doing certain complex transformations is very slow.

In [5]:
result = df.groupby('C_NATIONKEY').apply(lambda group: group.C_CUSTKEY.iloc[0] + group.C_CUSTKEY.mean())

## Let's switch the backend to python!

We pay a one-time cost of a few seconds to load the data into memory.

In [6]:
df.set_backend('python', inplace=True)

Transferring data from Snowflake to Python ...:   0%|          | 0/1 [00:00<?, ?it/s]

But now printing is extremely fast...

In [7]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...
3,30004,Customer#000030004,tkR93ReOnf9zYeO,23,33-870-136-4375,3308.55,AUTOMOBILE,ssly bold deposits. final req
4,30005,Customer#000030005,pvq4uDoD8pEwpAE01aesCtbD9WU8qmlsvoFav5,9,19-144-468-5416,-278.54,MACHINERY,ructions behind the pinto beans x-ra
...,...,...,...,...,...,...,...,...
149995,29996,Customer#000029996,BnZVGZiAgcEImNm9iD,7,17-536-308-8025,4035.17,FURNITURE,"ual instructions. bold, silent foxes nag blith..."
149996,29997,Customer#000029997,lTbDYXdQ74JctD UbRbXCqF2b8,9,19-631-777-4123,2015.90,HOUSEHOLD,eodolites detect slyly alongside of the quickl...
149997,29998,Customer#000029998,ZxxiuDruzi98CcymR,23,33-619-315-9722,-810.56,FURNITURE,xpress packages. accounts sleep carefully iron...
149998,29999,Customer#000029999,CuPA4UpgTCYiXrBrpiSO D,12,22-824-951-8333,3865.14,FURNITURE,eposits-- accounts haggle across the slyly per...


And so are most things we can imagine doing with the data.

In [8]:
result = df.groupby('C_NATIONKEY').apply(lambda group: group.C_CUSTKEY.iloc[0] + group.C_CUSTKEY.mean())
result

C_NATIONKEY
0     104809.648945
1     104849.461925
2     105879.897816
3     104446.823588
4     104679.314262
5     104906.427251
6     105917.388361
7     104694.312288
8     105173.743462
9     105037.669210
10    105448.956232
11    104112.225725
12    104829.006557
13    106009.666832
14    104419.615654
15    105264.778247
16    104199.408269
17    105125.565188
18    104305.006972
19    105746.410656
20    104615.581809
21    105924.703395
22    105530.506581
23    105457.153718
24    104223.723884
dtype: float64

When we're done with our transformations, we can write the results to Snowflake.

In [9]:
snow_result = result.rename('result').set_backend('snowflake')

Transferring data from Python to Snowflake ...:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
snow_result.to_snowflake('TEMP.MVASHISHTHA.RESULT', if_exists='replace', index=False)

# Use case 2: Filtering out most of a large table, then using python

## Setup

In this example, we read the table `SNOWPANDAS_PERF.TPCH_SF10.LINEITEM`, which is 60M rows and 1.3 GB.

In [11]:
df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF10.LINEITEM")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_WBRQGCOV7K' for snapshotting. DataFrame creation might take some time.


In [12]:
df

,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,L_COMMENT
0,58624485,982512,7522,7,14.0,22322.58,0.09,0.08,N,O,1997-05-11,1997-05-04,1997-05-31,DELIVER IN PERSON,REG AIR,uffily. blithely pending dep
1,58624486,1070302,20323,1,43.0,54706.75,0.06,0.02,N,O,1998-05-06,1998-05-16,1998-05-20,NONE,SHIP,above the sheaves haggle afte
2,58624486,552563,77569,2,24.0,38772.96,0.07,0.01,N,O,1998-06-22,1998-06-03,1998-06-27,TAKE BACK RETURN,AIR,"the regular, final dinos sleep furiously"
3,58624486,925493,521,3,2.0,3036.90,0.03,0.04,N,O,1998-05-08,1998-05-14,1998-05-16,TAKE BACK RETURN,FOB,iously bold deposits are
4,58624486,749590,24612,4,34.0,55745.04,0.05,0.07,N,O,1998-07-21,1998-04-27,1998-08-19,COLLECT COD,TRUCK,excuses shall haggle blithely. de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59986047,47852739,273848,23853,1,31.0,56476.73,0.02,0.08,N,O,1997-11-25,1998-01-31,1997-12-23,TAKE BACK RETURN,AIR,nding deposits are after the fluffily ironi
59986048,47852739,561695,61696,2,43.0,75536.81,0.01,0.05,N,O,1998-01-02,1998-02-05,1998-01-25,COLLECT COD,FOB,along the furiously even accounts. careful
59986049,47852739,396305,96306,3,43.0,60255.47,0.09,0.08,N,O,1997-12-30,1997-12-19,1998-01-15,TAKE BACK RETURN,SHIP,ncies sleep-- bravely enticing depen
59986050,47852739,692370,17377,4,14.0,19072.76,0.01,0.08,N,O,1998-01-19,1998-01-23,1998-02-12,NONE,REG AIR,regular ideas. bold platelet


The data is large, and pulling it all into pandas would take over a minute...

In [13]:
df.set_backend('python')

Transferring data from Snowflake to Python ...:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

But we only need to work with a sample of the data, so we sample 2% of the data.

In [14]:
filtered = df.sample(frac=0.02)

Now it's easier to fetch the data.

In [15]:
python_filtered = filtered.set_backend('python')

Transferring data from Snowflake to Python ...:   0%|          | 0/1 [00:00<?, ?it/s]

Now we can do a complex operation on the filtered subset of the data.

In [16]:
python_filtered.groupby('L_SHIPMODE').apply(lambda group: group.L_ORDERKEY.iloc[0] + group.L_ORDERKEY.mean())

L_SHIPMODE
AIR        8.861674e+07
FOB        8.861472e+07
MAIL       8.858904e+07
RAIL       8.868063e+07
REG AIR    8.863433e+07
SHIP       8.850255e+07
TRUCK      8.854551e+07
dtype: float64

Doing the same operation on the filtered data in Snowflake would take much longer...

In [17]:
filtered.groupby('L_SHIPMODE').apply(lambda group: group.L_ORDERKEY.iloc[0] + group.L_ORDERKEY.mean())


L_SHIPMODE
AIR        8.861674e+07
FOB        8.861472e+07
MAIL       8.858904e+07
RAIL       8.868063e+07
REG AIR    8.863433e+07
SHIP       8.850255e+07
TRUCK      8.854551e+07
dtype: float64